In [1]:
!pip install -q youtube-comment-downloader yt-dlp pandas tqdm

In [ ]:
# ── 설정 ──────────────────────────────────────────────────
TARGET_SIZE = 100_000  # target number of videos to collect
BATCH_SIZE = 500  # number of videos to fetch per query
MAX_DURATION = 60  # max duration in seconds (for shorts)

OUTPUT_DIR = "youtube_shorts_dataset"
ID_OUTPUT_PATH = f"{OUTPUT_DIR}/video_ids.csv"
# ──────────────────────────────────────────────────────────

In [3]:
# ── Step 0. 기존 ID 로드 (중복 방지) ─────────────────────
import pandas as pd
import os

os.makedirs(OUTPUT_DIR, exist_ok=True)

if os.path.exists(ID_OUTPUT_PATH):
    existing_df = pd.read_csv(ID_OUTPUT_PATH)
    seen = set(existing_df["video_id"].dropna().tolist())
    print(
        f"Loaded existing IDs: {len(seen)} -> skipping already collected videos"
    )
else:
    seen = set()
    print("No existing file found -> starting fresh")

print(
    f"Target: {TARGET_SIZE} / Current: {len(seen)} / Need to add: {max(0, TARGET_SIZE - len(seen))}"
)

Loaded existing IDs: 4022 -> skipping already collected videos
Target: 100000 / Current: 4022 / Need to add: 95978


In [4]:
# ── Step 1. Define search query list ─────────────────────
# Query diversity directly improves dataset diversity

GENERAL_QUERIES = [
    # viral
    "funny",
    "viral",
    "trending",
    "satisfying",
    # meme
    "meme",
    "funny meme",
    "relatable",
    # food
    "food",
    "cooking",
    "recipe",
    "asmr food",
    # animals
    "animals",
    "cute dog",
    "dog",
    "cute cat",
    "cat",
    # sports
    "sports",
    "basketball",
    "soccer",
    "gym",
    # games
    "gaming",
    "minecraft",
    "roblox",
    # beauty/fashion
    "makeup",
    "fashion",
    "outfit",
    # music
    "music",
    "dance",
    "singing",
    # education
    "did you know",
    "facts",
    "life hack",
    # etc
    "prank",
    "challenge",
    "travel",
    "nature",
    "science",
    "art",
    "diy",
    "comedy",
]

TRENDING_QUERIES = [
    # --------- treding.google.com filtering youtube
    # trending topics (2026)
    "half-life",
    "ai",
    "olympics",
    "bad bunny",
    # trending topics (2025)
    "kpop demon hunters",
    "soda pop",
    "67",
    # ---------
]

MADE_UP_QUERIES = [
    "fyp",
    "",  # for general shorts without specific keywords
]

BASE_QUERIES = GENERAL_QUERIES + TRENDING_QUERIES + MADE_UP_QUERIES
# BASE_QUERIES = MADE_UP_QUERIES

QUERIES = []
for q in BASE_QUERIES:
    QUERIES.append(f"{q} #shorts")

print(f"Total number of queries: {len(QUERIES)}")
print(
    f"{BATCH_SIZE} results per query → max candidates: {len(QUERIES) * BATCH_SIZE:,}"
)
print("Sample queries:", QUERIES[:5])

Total number of queries: 49
100 results per query → max candidates: 4,900
Sample queries: ['funny #shorts', 'viral #shorts', 'trending #shorts', 'satisfying #shorts', 'meme #shorts']


In [5]:
# ── Step 2. Collect IDs by iterating through queries ─────────
import yt_dlp

ydl_opts = {
    "quiet": True,
    "extract_flat": True,
    "skip_download": True,
}

newly_collected = []
need = TARGET_SIZE - len(seen)

for query in QUERIES:
    if len(newly_collected) >= need:
        break

    print(
        f"[{len(seen) + len(newly_collected)}/{TARGET_SIZE}] Searching: '{query}'"
    )

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            result = ydl.extract_info(
                f"ytsearch{BATCH_SIZE}:{query}", download=False
            )
        entries = result.get("entries", [])
    except Exception as ex:
        print(f"  ↳ error: {ex}")
        continue

    for e in entries:
        if len(newly_collected) >= need:
            break

        vid = e.get("id")
        if not vid or vid in seen:
            continue

        duration = e.get("duration")
        if duration is not None and duration > MAX_DURATION:
            continue

        seen.add(vid)
        newly_collected.append(
            {
                "video_id": vid,
                "title": e.get("title", ""),
                "duration": duration,
                "view_count": e.get("view_count"),
                "url": f"https://www.youtube.com/shorts/{vid}",
                "query": query,
            }
        )

print(f"\nCompleted: {len(newly_collected)} new videos collected")

[4022/100000] Searching: 'funny #shorts'
[4040/100000] Searching: 'viral #shorts'
[4082/100000] Searching: 'trending #shorts'
[4168/100000] Searching: 'satisfying #shorts'
[4206/100000] Searching: 'meme #shorts'
[4236/100000] Searching: 'funny meme #shorts'
[4240/100000] Searching: 'relatable #shorts'
[4334/100000] Searching: 'food #shorts'
[4380/100000] Searching: 'cooking #shorts'
[4406/100000] Searching: 'recipe #shorts'
[4458/100000] Searching: 'asmr food #shorts'
[4486/100000] Searching: 'animals #shorts'
[4486/100000] Searching: 'cute dog #shorts'
[4516/100000] Searching: 'dog #shorts'
[4544/100000] Searching: 'cute cat #shorts'
[4576/100000] Searching: 'cat #shorts'
[4608/100000] Searching: 'sports #shorts'
[4638/100000] Searching: 'basketball #shorts'
[4658/100000] Searching: 'soccer #shorts'
[4672/100000] Searching: 'gym #shorts'
[4708/100000] Searching: 'gaming #shorts'
[4758/100000] Searching: 'minecraft #shorts'
[4810/100000] Searching: 'roblox #shorts'
[4826/100000] Search

In [6]:
# ── Step 3. Append new data to existing CSV ───────────────────────
new_df = pd.DataFrame(newly_collected)

if os.path.exists(ID_OUTPUT_PATH) and len(newly_collected) > 0:
    new_df.to_csv(ID_OUTPUT_PATH, mode="a", header=False, index=False)
    print(f"Appended {len(new_df)} new videos → {ID_OUTPUT_PATH}")
elif len(newly_collected) > 0:
    new_df.to_csv(ID_OUTPUT_PATH, index=False)
    print(f"Saved {len(new_df)} new videos → {ID_OUTPUT_PATH}")
else:
    print("No new videos collected")

total = pd.read_csv(ID_OUTPUT_PATH)
print(f"Total accumulated: {len(total)} videos")
total.tail()

Appended 796 new videos → youtube_shorts_dataset/video_ids.csv
Total accumulated: 4818 videos


,video_id,title,duration,view_count,url,query
4813,eFOf2i5Jeu8,WOW!!! So Satisfying! 😍 Ice Ball with Doll! #s...,8.0,7691.0,https://www.youtube.com/shorts/eFOf2i5Jeu8,#shorts
4814,H2UC5EMxh5w,Redemption Mailday From Topps! What Did We Get...,54.0,734.0,https://www.youtube.com/shorts/H2UC5EMxh5w,#shorts
4815,u6ls0ITaxYI,Realme GT 7T Video Test 4k #shorts,42.0,0.0,https://www.youtube.com/shorts/u6ls0ITaxYI,#shorts
4816,ftF2JZHvI9Q,#shorts ОБНОВЛЕНИЕ в ЦУНАМИ и в Steal a Brainr...,NaN,NaN,https://www.youtube.com/shorts/ftF2JZHvI9Q,#shorts
4817,VYbYL1eXIgQ,Suzuki Gixxer SF 🔥#suzuki #gixxer #bikelife #b...,12.0,641.0,https://www.youtube.com/shorts/VYbYL1eXIgQ,#shorts
